In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

from datetime import datetime
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../data/1_shark_limpio.csv", encoding='latin-1')

In [3]:
df.columns = df.columns.str.strip()

df = df.drop(["Sex","Species" ,"Age", "Type", "Country", "Area", "Location", "Activity", "Name", "Injury", "Fatal (Y/N)","Time", "Investigator or Source","pdf","href formula","href","original order"],axis = 1)

# Limpieza de fecha

In [4]:
df.head()

,Case Number,Date,Year,Case Number.1,Case Number.2
0,2018.06.25,25-Jun-2018,2018.0,2018.06.25,2018.06.25
1,2018.06.18,18-Jun-2018,2018.0,2018.06.18,2018.06.18
2,2018.06.09,09-Jun-2018,2018.0,2018.06.09,2018.06.09
3,2018.06.08,08-Jun-2018,2018.0,2018.06.08,2018.06.08
4,2018.06.04,04-Jun-2018,2018.0,2018.06.04,2018.06.04


In [5]:
def sacar_fecha(date_str):
    # Define a regex pattern to match the components with different delimiters
    pattern = r'(\d{4})[/.-](\d{2})[/.-](\d{2})'

    # Use the regex pattern to match and extract the components
    matches = re.search(pattern, date_str)

    if matches:
        year = matches.group(1)
        month = matches.group(2)
        day = matches.group(3)
        return year, month, day
    else:
        return 0, 0, 0

In [6]:
df[['Y_casen1', 'M_casen1', 'D_casen1']] = df['Case Number.1'].apply(sacar_fecha).apply(pd.Series)

In [7]:
df[['Y_casen2', 'M_casen2', 'D_casen2']] = df['Case Number.2'].apply(sacar_fecha).apply(pd.Series)

In [8]:
#df = df.drop(["Case Number.1", "Case Number.2"], axis=1)

In [9]:
df.head()

,Case Number,Date,Year,Case Number.1,Case Number.2,Y_casen1,M_casen1,D_casen1,Y_casen2,M_casen2,D_casen2
0,2018.06.25,25-Jun-2018,2018.0,2018.06.25,2018.06.25,2018,06,25,2018,06,25
1,2018.06.18,18-Jun-2018,2018.0,2018.06.18,2018.06.18,2018,06,18,2018,06,18
2,2018.06.09,09-Jun-2018,2018.0,2018.06.09,2018.06.09,2018,06,09,2018,06,09
3,2018.06.08,08-Jun-2018,2018.0,2018.06.08,2018.06.08,2018,06,08,2018,06,08
4,2018.06.04,04-Jun-2018,2018.0,2018.06.04,2018.06.04,2018,06,04,2018,06,04


Establecemos las funciones para de la columna date sacar año, mes y día.

In [10]:
def extract_year(date_string):
    pattern = r"(\d{4})"
    match = re.search(pattern, date_string)
    if match:
        return match.group(1)
    else:
        return "0"


# Apply the extract_year function to the Date column
df['Year'] = df['Date'].apply(extract_year)
df['Year'] = df['Year'].astype(int)

In [11]:
def extract_month(date_string):
    match = re.search(r'(\d{2}-[A-Za-z]{3}-\d{4})', date_string)
    if match:
        date_part = match.group(1)
        date_object = datetime.strptime(date_part, "%d-%b-%Y")
        return date_object.month
    else:
        return "0"

# Apply the function to the 'Date' column
df['Month'] = df['Date'].apply(extract_month)
df['Month'] = df['Month'].astype(int)

In [12]:
def extract_day(date_string):
    pattern = r"(\d{2})"
    match = re.search(pattern, date_string)
    if match:
        return match.group(1)
    else:
        return "0"


# Apply the extract_year function to the Date column
df['Day'] = df['Date'].apply(extract_day)
df['Day'] = df['Day'].astype(int)

In [13]:
#df = df.drop(["Date"], axis = 1)

Ahora tenemos tres columnas en las que presentamos los datos de la fecha. A continuación desecharemos la columna "Date" ya que no tiene sentido conservarla. Despues revisaremos los valores creados

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6302 entries, 0 to 6301
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Case Number    6301 non-null   object
 1   Date           6302 non-null   object
 2   Year           6302 non-null   int64 
 3   Case Number.1  6302 non-null   object
 4   Case Number.2  6302 non-null   object
 5   Y_casen1       6302 non-null   object
 6   M_casen1       6302 non-null   object
 7   D_casen1       6302 non-null   object
 8   Y_casen2       6302 non-null   object
 9   M_casen2       6302 non-null   object
 10  D_casen2       6302 non-null   object
 11  Month          6302 non-null   int64 
 12  Day            6302 non-null   int64 
dtypes: int64(3), object(10)
memory usage: 640.2+ KB


In [15]:
#df[df.isna().any(axis=1)]

Vemos que solo existe un NaN en el Data Frame en la columna Case Number. Observando la fila vemos que podemos rellenar este valor con los datos de Case Number.1 o Case Number.2

In [16]:
df['Case Number'].fillna(df['Case Number.1'], inplace=True)

In [17]:
df.loc[5488]

Case Number              1905.09.06.R
Date             Reported 06-Sep-1905
Year                             1905
Case Number.1            1905.09.06.R
Case Number.2            1905.09.06.R
Y_casen1                         1905
M_casen1                           09
D_casen1                           06
Y_casen2                         1905
M_casen2                           09
D_casen2                           06
Month                               9
Day                                 6
Name: 5488, dtype: object

A continuación pasamos a revisar los datos creados para verificar su eficacia

Vemos que hay 19 valores donde el año es 0. Tratamos de sobreescribirlos con la información en el resto de columnas.

In [18]:
df[df["Year"] == 0]

,Case Number,Date,Year,Case Number.1,Case Number.2,Y_casen1,M_casen1,D_casen1,Y_casen2,M_casen2,D_casen2,Month,Day
4644,1944.07.22,22-Jul-144,0,1944.07.22,1944.07.22,1944,07,22,1944,07,22,0,22
4802,1939.00.00.e,Woirld War II,0,1939.00.00.e,1939.00.00.e,1939,00,00,1939,00,00,0,0
4838,1937.09.26.R,Reported 26-Sep-t937,0,1937.09.26.R,1937.09.26.R,1937,09,26,1937,09,26,0,26
6174,0500.00.00,Circa 500 A.D.,0,0500.00.00,0500.00.00,0500,00,00,0500,00,00,0,50
6175,0077.00.00,77 A.D.,0,0077.00.00,0077.00.00,0077,00,00,0077,00,00,0,77
6176,0005.00.00,Ca. 5 A.D.,0,0005.00.00,0005.00.00,0005,00,00,0005,00,00,0,0
6177,0000.0214,Ca. 214 B.C.,0,0000.0214,0000.0214,0,0,0,0,0,0,0,21
6178,0000.0336,Ca. 336.B.C..,0,0000.0336,0000.0336,0,0,0,0,0,0,0,33
6179,0000.0493,493 B.C.,0,0000.0493,0000.0493,0,0,0,0,0,0,0,49
6180,0000.0725,Ca. 725 B.C.,0,0000.0725,0000.0725,0,0,0,0,0,0,0,72


In [19]:
df.at[4644, 'Year'] = 1944
df.at[4802, 'Year'] = 1939
df.at[4838, 'Year'] = 1937

df.at[4644, 'Month'] = 7
df.at[4802, 'Month'] = 0
df.at[4838, 'Month'] = 9

df.at[4644, 'Day'] = 22
df.at[4802, 'Day'] = 0
df.at[4838, 'Day'] = 26

In [20]:
#indices = df[df['Final_year'] == 0].index.tolist()
#indices

In [21]:
#df = df.drop(indices)

Nos sacamos valores que no interesan. Previos a la IIWW. Inconclusos o sin suficientes datos. A continuación revisamos dias y años

# Checkpoint

Ahora que hemos creado nuevas columnas y tenemos varios datos buscaremos cruzar las tablas para unificar el df

In [22]:
df.shape

(6302, 13)

In [23]:
df.head(0)

,Case Number,Date,Year,Case Number.1,Case Number.2,Y_casen1,M_casen1,D_casen1,Y_casen2,M_casen2,D_casen2,Month,Day


In [24]:
df[['Y_casen1', 'M_casen1', 'D_casen1']]=df[['Y_casen1', 'M_casen1', 'D_casen1']].astype(int)

In [25]:
df[['Y_casen2', 'M_casen2', 'D_casen2']]=df[['Y_casen2', 'M_casen2', 'D_casen2']].astype(int)

In [26]:
# Function to apply the conditions
def apply_conditions(row):
    original = row['Year']
    casen1 = row['Y_casen1']
    casen2 = row['Y_casen2']

    if original == casen1:
        return casen1
    elif original == casen2:
        return casen2
    else:
        return original

# Apply the function to create a final 'yaer' column
df['Final_year'] = df.apply(apply_conditions, axis=1)

In [27]:
# Function to apply the conditions
def apply_conditions(row):
    original = row['Month']
    casen1 = row['M_casen1']
    casen2 = row['M_casen2']

    if original == casen1:
        return casen1
    elif original == casen2:
        return casen2
    else:
        return original

# Apply the function to create a final 'yaer' column
df['Final_month'] = df.apply(apply_conditions, axis=1)

In [28]:
# Function to apply the conditions
def apply_conditions(row):
    original = row['Day']
    casen1 = row['D_casen1']
    casen2 = row['D_casen2']

    if original == casen1:
        return casen1
    elif original == casen2:
        return casen2
    else:
        return original

# Apply the function to create a final 'yaer' column
df['Final_day'] = df.apply(apply_conditions, axis=1)

In [29]:
df.head(0)

,Case Number,Date,Year,Case Number.1,Case Number.2,Y_casen1,M_casen1,D_casen1,Y_casen2,M_casen2,D_casen2,Month,Day,Final_year,Final_month,Final_day


In [30]:
df_fechas = df[["Final_year", "Final_month", "Final_day"]]

In [31]:
df_fechas.shape

(6302, 3)

In [32]:
df_fechas.to_csv("../data/2_shark_fechas.csv")